<a href="https://colab.research.google.com/github/gretiere545/corpus/blob/main/corpus_maker_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fabrique des Corpus ASAMLA

## Phase 1 : Initialisation d'un nouveau Corpus

In [25]:
!export PYTHONIOENCODING=utf8
!pip install gspread-formatting
import pandas as pd
import numpy as np
import base64
import uuid
import zlib
import binascii
import random
import os
import json
import secrets
pd.set_option("display.width",1000)
# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_formatting import *
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA


In [27]:
def make_unique_id():
  return str(uuid.uuid4())[:8]

In [28]:
def format_feuille(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)
  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:B', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=2)
  set_column_width(ws, 'A', 100)
  set_column_width(ws, 'B', 400)
  set_column_width(ws, 'C:', 600)

Référentiels Corpus de travail

In [8]:
# Corpus Français Médical Vaccination
corpus_fr_med_vac = "asamla-corpus_francais_med_vac-v1.2"
corpus_fr_med_vac_uri = "https://docs.google.com/spreadsheets/d/1-2SDHCFR5JOHCc4MiYSMo4V8yG7kfHbpIKV3q9ci6wE"
# Corpus Français Médical Médecine Générale
corpus_fr_med_gen = "ASAMLA - Corpus Médical - Médecine générale-v1.0"
corpus_fr_med_gen_uri = "https://docs.google.com/spreadsheets/d/1M3CFSpAGII25qCl0apYrRPX-DmpdxDFLLB_Ob-MX8DQ"

In [29]:
# Corpus Toutes Langues Médical Vaccination
corpus_trad_med_vac = "ASAMLA - Corpus Médical Vaccination-v2.0"
corpus_trad_med_vac_uri = "https://docs.google.com/spreadsheets/d/1CclzYfFCW4srA3Lq_np2LpSrxj84JpcbzytL449DH8E"
# Corpus Toutes Langues Médical Médecine Générale
corpus_trad_med_neu = "ASAMLA - Corpus Médical Médecine Générale-v1.0"
corpus_trad_med_neu_uri = "https://docs.google.com/spreadsheets/d/1zB7hvjr2HufHcVgcJfEtrUUmR75wU5OCiL-_d4oDtTU"

Ouverture de la base centrale (repository) : base se trouvant sur gdrive atos.net

In [30]:
# Ouverture de la Sheet Centrale (corpus_central_base)
sheet_central = 'https://docs.google.com/spreadsheets/d/1L8YB1aXHUJwUE9AE6xyn_xMHalinGR335Q7lntwbu1U'
wb_central = gc.open_by_url(sheet_central)
# onglet Médical Vaccination
corpus_trad_med_vac_onglet = "med_vac_synthese"
# onglet Médical Médecine Générale
corpus_trad_med_gen_onglet = "med_gen_synthese"

Chargement du Corpus FR : à ce stade, il ne contient que des expressions en français

In [31]:
def get_cc_config(cfg):
  vk_lang_dict = []
  try:
    with open(cfg) as vk_dict:
        vk_lang_dict = json.load(vk_dict)
  except BaseException as e:
    print(e)
  return vk_lang_dict

def set_cc_config (vk_lang_dict, cfg):
  # sauvegarde du fichier de paramétrage
  with open(cfg, 'w') as fp:
      json.dump(vk_lang_dict, fp)  

In [32]:
lang_config = 'med_vac_synthese.json'
vk_lang_dict = get_cc_config(lang_config)

In [33]:
uri = corpus_fr_med_gen_uri

In [34]:
def narrow_no_break_space (s):
  s = " ".join(s.split())   # on vire tous les espaces en trop
  find = r'(\s*\?$)'        # on recherche le ? précédé ou pas d'espace \s*
  replace = u'\u202F'+ r'?'
  s = re.sub(find, replace, s)    # on remplace par un blanc insécable + le ?
  return s

2021 10 05 : 
Suppression du Peul, remplacé par Allemand

Fonction d'attribution d'id unique pour chaque expression à l'intérieur d'un corpus

In [35]:
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
t_corpus = wb.get_worksheet(0)
data_t_corpus = t_corpus.get_all_values()
df_corpus = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0])

In [40]:
df_corpus

,uid,français,traduction
0,03bb6e86,"Bonjour, comment vous appelez-vous ?",
1,38a24ddb,Quel est le motif de votre visite ?,
2,ba312208,Quel âge avez-vous ?,
3,192bc4c8,Vous venez de quel pays ?,
4,955f030c,Buvez-vous de l’alcool ?,
...,...,...,...
78,50460084,Saignement,
79,af9faaa5,Purulent,
80,1ffe71ca,Fatigue,
81,6fa427fc,angine,


In [37]:
# SI L'état de l'expression est VALIDE on ajoute le hash qui sert de clé
df_corpus['uid'] = df_corpus[['uid','état']].apply(lambda x: x[0] if (x[0]!="" and x[1]=="validé") else make_unique_id() if (x[0]=="" and x[1]=="validé") else "", axis=1)
# exemple de bout de code pour accéder à un enregistrement par la clé
# df_corpus.loc[df_corpus['uid']=='g06g6WJ6ROb8-tWX2TlVtwvKY3MdWg']['coded_expression'].apply(lambda x:decode_expression(x))

In [38]:
# on élimine toutes les lignes sans expression
df_corpus["expression"].replace('', np.nan, inplace=True)
df_corpus = df_corpus[df_corpus['expression'].notna()]
# trim des expressions
df_corpus['français'] = df_corpus['expression'].apply(lambda x:x.strip())
df_corpus['français'] = df_corpus['français'].apply(lambda x:narrow_no_break_space(x))   

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [39]:
# on ne conserve que les colonnes utiles
df_corpus = df_corpus[['uid','français']]
# on ajoute la colonne pour la traduction
df_corpus['traduction'] = ''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Sauvegarde du corpus FR avec l'uid

In [41]:
# sauvegarde du dataframe dans la table wb_central de la feuille gsheet
set_with_dataframe(t_corpus, df_corpus)


In [ ]:
df_corpus = df_corpus.loc[df_corpus['uid']!=""]

#### Déploiement du corpus français dans un corpus de traduction : 1 langue = 1 onglet

In [42]:
uri = corpus_trad_med_neu_uri
#ouverture de l'onglet Corpus dans un DF
wb = gc.open_by_url(uri)
#pour chaque corpus, on créé un onglet si besoin
for index, item in enumerate(vk_lang_dict):
  #df_corpus_all.append (df_corpus)
  nom_onglet = item['language']
  # si l'onglet existe déjà, on le supprime ?
  try:
    ws = wb.worksheet(nom_onglet)
    wb.del_worksheet(ws)  
  except:
    print ("Onglet inexistant !")
  wb.add_worksheet(nom_onglet, 1, 1)
  export_sheet = wb.worksheet(nom_onglet)
  set_with_dataframe(export_sheet, df_corpus)
  format_feuille(wb, nom_onglet)    